## Imports, installation, authentication, initialization

Get your notebook set up!


In [4]:
!pip install geemap

import ee
import numpy as np
import geemap.eefolium as geemap
import pandas as pd

ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=0RFVWYUUk3-kgAoHxr4DtM2j4kYEYtOrG4-F3hrQvUA&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g72pTPns6Sxitk_0jRWDtRQ0fumLZCuJ_Z-y0gsXh-sF7y2x6ar5hs

Successfully saved authorization token.


# Part 1: Cloud Masking

First, we will create a few functions.
We will create a cloud mask using the pixel_qa band for Landsat.

To understand how data is stored in the pixel_qa band check out the Landsat-457 information [here](https://www.usgs.gov/core-science-systems/nli/landsat/landsat-sr-derived-spectral-indices-pixel-quality-band) 

or for Landsat-8 [Landsat Surface Reflectance Code Product Guide](https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_C1-LandSurfaceReflectanceCode-LASRC_ProductGuide-v3.pdf)





In [6]:
def cloudMaskL457(image):
  qa = image.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloud = qa.bitwiseAnd(1 << 5) \
                  .And(qa.bitwiseAnd(1 << 7)) \
                  .Or(qa.bitwiseAnd(1 << 3))
  # Remove edge pixels that don't occur in all bands
  mask2 = image.mask().reduce(ee.Reducer.min())
  return image.updateMask(cloud.Not()).updateMask(mask2)

In [8]:
# Let's apply the cloud mask to a Landsat-5 image to see what it does

# This asset is a polygon of the Ottawa Census Metropolitan Area
OttawaPoly = ee.FeatureCollection('users/koreenmillard/OttawaCMA_dissolved')  # I've made this asset publically available, but you can change it to your own file if you want

lan5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                  .filter(ee.Filter.Or(
                  ee.Filter.And(ee.Filter.date('2007-06-15', '2007-08-15')),
                  ee.Filter.And(ee.Filter.date('2008-06-15', '2008-08-15')),
                  ee.Filter.And(ee.Filter.date('2009-06-15', '2009-08-15')),
                  ee.Filter.And(ee.Filter.date('2010-06-15', '2010-08-15')))) \
                  .filterBounds(OttawaPoly) \
                  .map(cloudMaskL457)



In [9]:
# Let's print some information what we just produced
print("lan5 is an", lan5.name())
print('\n')

# Print out all the band ids for an image json
def printImageBands(image):
  print('Bands of example image:')
  for band in range(len(image['bands'])):
    print(image['bands'][band]['id'])
  print('\n')

# Print the size of a json image collection
def printCollectionSize(collection):
  print('Size of image collection:')
  print(len(collection['features']))
  print('\n')

# Get an image json object from a json image collection based on an index value
def getImageFromCollection(collection, index=0):
  return collection['features'][index]
##end functions

# Print section
# First, we need to use the getInfo function to get all the information about our image Collection into json format  
stack_json = lan5.getInfo()

# Then, we can run our neat print functions on new variable
# printCollectionSize tells us how many images are in our image collection
printCollectionSize(stack_json)

# If we want to look at a specific image in the collection, we need to first grab an image from our json variable and put it in a new variable
img_json = getImageFromCollection(stack_json)

# Then we can look at what bands are in this image
printImageBands(img_json)

lan5 is an ImageCollection


Size of image collection:
40


Bands of example image:
B1
B2
B3
B4
B5
B6
B7
sr_atmos_opacity
sr_cloud_qa
pixel_qa
radsat_qa




In [10]:
# Let's visualize the results
Map = geemap.Map(width = 1500, height = 1500)

# We need to create a visualization parameter variable  
# Whis will tell GEE which bands to display, and what colour stretch to use
# Set the L8 true colour vis params
L5_TC_vis = {
  'bands': ['B3', 'B2', 'B1'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4,
}

SHPViz = {
    'color': 'red' 
    }

Map.addLayer(lan5 ,L5_TC_vis,'lan5')
Map.addLayer(OttawaPoly, SHPViz, 'OttawaPoly')
Map.addLayerControl()
Map.centerObject(OttawaPoly,10)

Map

You'll notice that this just plopped all of the images on the map.

Let's write a function to select just one of the images in the collection.

In [13]:
# Get an image from an image collection based on the index of the image
# Note: This is working with images and image collections, not json
# index = 0 is set so that it defaults to the first image if you don't set an imageID when you call this function
def getImageByIndex(collection, index=0):  

  # Convert the image collection to a list
  listOfImages = collection.toList(collection.size())

  # Return an image based on its index
  # This must be cast to the ee.Image type
  return ee.Image(listOfImages.get(index))

In [14]:
secondImage = getImageByIndex(lan5, 1)

print("lan5 is an", lan5.name())
print("secondImage is a", secondImage.name())


lan5 is an ImageCollection
secondImage is a Image


In [15]:
Map = geemap.Map(width = 1500, height = 1500)

#use the same visualization parameters as before
Map.addLayer(secondImage ,L5_TC_vis,'lan5')
Map.addLayer(OttawaPoly, SHPViz, 'OttawaPoly')
Map.addLayerControl()
Map.centerObject(OttawaPoly,10)

Map

# Part 2: Produce a quality mosaic

A quality mosaic let's you specify the rule you want to use to pick the "best" pixel in your image collection.

Clouds are masked first, but in many cases pixels will still have many pixels in the stack that contain cloud free results.  So, you need to determine which pixel to use.  

One common way is using the "greenest" pixel (or the pixel with the highest NDVI value).

We will write a function to do that.  

In [17]:
# This function returns the cloudmasked images with an NDVI band added. The NDVI band will be used to determine the quality of a pixel - for Landsat5 (edit for other sensors)
def addQualityBands(l5image):
  return cloudMaskL457(l5image) \
                  .addBands(l5image.normalizedDifference(['B4', 'B3']).rename('ndvi')) \
                  .addBands(l5image.metadata('system:time_start')) 

Mosaic the images using NDVI as the "quality band" and display them.

In [18]:
# Here we will select some images based on date and location, apply the cloudmask and select the "best available pixel"
# Here I am making a mosaic across multiple years (any image from 2007 - 2010 in the summer)

lan5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
                  .filter(ee.Filter.Or(
                  ee.Filter.And(ee.Filter.date('2007-06-15', '2007-08-15')),
                  ee.Filter.And(ee.Filter.date('2008-06-15', '2008-08-15')),
                  ee.Filter.And(ee.Filter.date('2009-06-15', '2009-08-15')),
                  ee.Filter.And(ee.Filter.date('2010-06-15', '2010-08-15')))) \
                  .filterBounds(OttawaPoly) \
                  .map(addQualityBands) \
                  .qualityMosaic ('ndvi') \
                  .float()


clippedLan5 = lan5.clipToCollection(OttawaPoly)

In [19]:
# Visualize all three layers (image1, image2, mosaicked) on the map
Map = geemap.Map()
visParams = {
       'bands': ['B3', 'B2', 'B1'], # Change these bands depending on what you want to see
       'min': 0,                    # These values are the min/max values for an L5_SR image
       'max': 3000,
       'gamma': 1.4,                # Gamma applies a correction/stretch to each band.  You can specify one value for each band, or one for all
     }

Map.addLayer(clippedLan5, visParams, 'L5 Ottawa Summer Mosaic')
Map.addLayerControl()
Map.centerObject(OttawaPoly,8) # Center on the polygon at scale "8"
Map

# Part 3: Get Zonal Statistics from our Mosaic

Let's get the mean value of each band for each census tract in Ottawa.

In [20]:
# Let's bring in an assett that contains all of the CMA boundaries
# This asset contains 227 polygons for the city of Ottawa
OttawaCMA = ee.FeatureCollection('users/koreenmillard/OttawaCMA_UniqueID')


CT_mean = clippedLan5.reduceRegions(**{
  'collection': OttawaCMA,
  'reducer': ee.Reducer.mean(),
  'scale': 30 
})

# let's try to print some info about this dataset we just created
ee.Feature(CT_mean.first()).select(lan5.bandNames()).getInfo()

{'geometry': {'coordinates': [[[-75.88021014418736, 45.43729937750564],
    [-75.88018102613958, 45.437166923281474],
    [-75.88018231210671, 45.43702181881723],
    [-75.8801531944667, 45.436889365392744],
    [-75.88015144829285, 45.43675281309101],
    [-75.8800944659805, 45.436342874719045],
    [-75.87998551445345, 45.435492082503245],
    [-75.87997015901988, 45.43533947231982],
    [-75.87995194407914, 45.43517720458312],
    [-75.87994670730184, 45.43508546552302],
    [-75.87993395924877, 45.43499680885814],
    [-75.87991874242917, 45.43482606027078],
    [-75.87988397763213, 45.43465629498074],
    [-75.87986879597801, 45.43448547880305],
    [-75.87983706415557, 45.434307163945284],
    [-75.87954171828301, 45.43195129960417],
    [-75.87929337852319, 45.43026238142789],
    [-75.87923190940242, 45.429847101662695],
    [-75.87889552042881, 45.42767457247449],
    [-75.87751425933884, 45.42778355406371],
    [-75.87682584824427, 45.427849811251534],
    [-75.87677738749781

In [21]:
# that wasn't all that easy to read.  Let's covert it to a pandas dataframe
df = geemap.ee_to_pandas(CT_mean)
df.head() #the head function just displays the first few records 

,B1,B2,B3,B4,B5,B6,B7,CMANAME,CMAPUID,CMATYPE,CMAUID,CTNAME,CTUID,PRNAME,PRUID,UniqueID,ndvi,pixel_qa,radsat_qa,sr_atmos_opacity,sr_cloud_qa
0,460.079170,712.982259,556.610706,3194.987349,1722.674528,2950.679161,919.363758,Ottawa - Gatineau (partie du Québec / Quebec p...,24505,B,505,0822.05,5050822.05,Quebec / Québec,24,1,0.701773,67.181608,0.000000,197.617013,0.233559
1,379.257844,610.227347,440.891527,3222.409278,1611.824155,2946.575012,764.665184,Ottawa - Gatineau (partie du Québec / Quebec p...,24505,B,505,0822.06,5050822.06,Quebec / Québec,24,2,0.700560,66.549876,0.000000,192.050255,1.987961
2,766.107420,1005.659285,936.676970,2550.802969,1775.845655,3007.164523,1256.436042,Ottawa - Gatineau (partie du Québec / Quebec p...,24505,B,505,0506.00,5050506.00,Quebec / Québec,24,14,0.461039,66.800331,0.000000,286.841870,0.190327
3,429.023508,615.085743,486.958674,2452.668472,1283.511673,2953.078404,694.582938,Ottawa - Gatineau (partie du Québec / Quebec p...,24505,B,505,0507.00,5050507.00,Quebec / Québec,24,15,0.604634,66.912678,0.000000,247.016688,4.858392
4,858.599979,1113.068426,1067.104139,2424.321740,1904.549376,3007.523640,1448.655428,Ottawa - Gatineau (partie du Québec / Quebec p...,24505,B,505,0508.00,5050508.00,Quebec / Québec,24,16,0.392226,66.981141,0.019128,307.729114,0.159388


In [22]:
# Mounting your google drive
from google.colab import drive
drive.mount('/content/drive')

df.to_csv('/content/drive/My Drive/CSRS2021_GEEWorkshop_ForStudents/OttawaCMA_ZonalStatistics_Landsat.csv')

Mounted at /content/drive
